# Introduction
> Part 1 of the mangoes_blog project

- branch: master
- toc: true 
- badges: false
- comments: false
- sticky_rank: 1
- author: Huon Fraser
- categories: [mangoes]

Deep learning for tabular data is experiencing a slight renaissance. Several methods have emerged that can compete to boosting methods (such as [XGBoost](https://xgboost.readthedocs.io/en/stable/)) in favourable conditions. NIR-spectrometry is a branch of tabular data that typically has two properties, large dimensionality and [multicollinearity](https://en.wikipedia.org/wiki/Multicollinearity), which in theory make it suited to deep networks. Multilayer Perceptron networks have been used for decades, and recently CNN architectures have been proposed, however, linear regression models and domain related preprocessing are still the predominant technique. For a detailed review of the NIR-spectra I defer to [Anderson and Walsh](https://opg.optica.org/jnirs/abstract.cfm?uri=jnirs-30-1-3).


Applications in Industry use huge amounts of test instances, the horticuture industry for iinstance automated grading in horticure packhouses. These analysis rely on prior lab analysis for labelling data. As this analysis is relatively expensive (especially for large amounts of data), most datasets are proprietry. Enter the [Mangoes dataset](https://data.mendeley.com/datasets/46htwnp833/1) (Anderson et al.), a large NIR-spectroscopy dataset for estimating the relationship between spectra and the dry matter, a measure of maturity of mango fruit.


This is first part of a series comparing a range of deep networks to classical approaches the Mangoes data set. The advantage of the Mangoes dataset is that models (and performance metrics) are publicly available for both classical ([Anderson et al.](https://www.researchgate.net/publication/342056149_Achieving_robustness_across_season_location_and_cultivar_for_a_NIRS_model_for_intact_mango_fruit_dry_matter_content)) and deep network ([Mishra and Passos](https://analyticalsciencejournals.onlinelibrary.wiley.com/doi/full/10.1002/cem.3367)) approaches. We can start by (re-)producing similar work and once results are inline we can veer off. The plan is to start with simple linear models and gradually adding complexity. Likel we will do the same for deep networks. Currently the following  parts are sketched in, with the plan for deep networks being less clear:

1) Introduction - Exploratory data analysis and a first linear model.
2) Evaluation - Setup our cross validation and search methods.
3) Spectral Regressions - Cover regression techniques commonly used for spectral applications.
4) Spectral Preprocessing - Cover preprocessing techniques commonly used for spectral applications.
5) Black Box tabular - Compare results to ensembles like RandomForest and XGGoost.
6) Deep Networks ... 


This notebook introduces the Mangoes data set and performs expoloratory data analysis. These are done in a literate programming style with text, code and outputs alongside each other. We use the [nbdev](https://www.fast.ai/2019/12/02/nbdev/) library, code blocks marked with #export are reused in future notebooks, allowing us to incrementally build a library of tools.  Occasionally code segments are deemed too complex/large for a notebook and these will be available in the github repository.

## Setup 

We begin with imports and library settings which we will use for all notebooks. 

In [1]:
#collapse-hide
import pathlib
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.pipeline import Pipeline

np.random.seed(123)

import warnings
warnings.filterwarnings('ignore')

ImportError: C extension: numpy.core.multiarray failed to import not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext --inplace --force' to build the C extensions first.

Next we define methods to parse the Mangoes data set. We write a method to load the data (load_mangoes). Training and test splits are provided in the data (for reproducability) and we write a method to seperate these (train_test_split). We then write a method to seperate the spectra data, the target, and other variables into seperate dataframes, with the option to limit the spectra to certain wavelengths. More details about these partitions will be provided below.

In [ ]:
def load_mangoes():
    mangoes = pd.read_csv("../data/mangoes_raw.csv")

    unique_spectra = mangoes['DM'].unique()
    fruit_id_dict = {u:i for i,u in enumerate(unique_spectra)}
    mangoes['Fruit_ID'] = mangoes['DM'].apply(lambda x: fruit_id_dict[x])
    return mangoes

def train_test_split(data):
    train_inds = np.logical_not(data['Set']=='Val Ext')
    test_inds = data['Set']=='Val Ext'

    train_data = data[train_inds]
    test_data = data[test_inds]
    
    return train_data, test_data


def X_y_cat(data,min_X=285,max_X=1200):
    cat_vars=['Set','Season','Region','Date','Type','Cultivar','Pop','Temp','Fruit_ID']
    y_vars = ['DM']
    X_vars = [i for i in data.columns if (not i in y_vars) and (not i in cat_vars)]
    X_vars = [i for i in X_vars if (int(i)>= min_X) and (int(i)<= max_X)]
    return data[X_vars], data[y_vars], data[cat_vars]

Running these methods we see that the dataset has 11691 instances, 10243 of which are marked for training and 1448 for testing.

In [ ]:
mangoes = load_mangoes()    
train_data,test_data = train_test_split(mangoes)

train_X, train_y, train_cat= X_y_cat(train_data)
test_X, test_y, test_cat = X_y_cat(test_data)
nrow,ncol=train_X.shape

print(f'Data shape: {mangoes.shape}')
print(f'Train data shape: {train_data.shape}')
print(f'Test data shape: {test_data.shape}')

Taking a step back, below we show a sample of the entire dataset. The first 8 columns are metadata and the 9th, DM, is the target variable. The remaining columns are the spectras, correspnding to readings over 3nm intervals from 285-1200nm.   Multiple readings are made for each mango fruit. Each spectra is included twice, corresponding to two spectra readings. Several fruit are read more than twice, for example multiple readings are made at different temperatures. Above we assigned a 'Fruit_ID' column based on unique values of DM. This isn't pefect, different fruit may have the same DM at the precision present in the dataset, or fruit may have different DM values but may be a useful proxy.

In [ ]:
print(mangoes.head())

We also show a selection of spectra from the training portion of the dataset. Missing values in the dataset are coded as 0. Values are missing at each end of the spectrum and are coded as 0. Several columns are completely 0, indicated by the flat lines at each of the below figure. Other columns are partially 0, likely due to different instruments being used to measure different groups of samples. The range of data which includes no missing data is 513-1050nm. Indidently, this is the range that cuts of much of the noise seen at each end. 

In [ ]:
ax=train_X.mean().plot(label='Mean',legend=True)
for i in range(0,4):
    train_X.iloc[i,:].plot(label=i,legend=True)

We also show the distributuon of the target (DM) for the non-test partitions of the data. This is fairly symmetrical and no outliers are present. 

In [ ]:
train_y.plot.hist(bins=50,density=True)

Anderson et al. used the 684-990nm range for their analysis, so for the sake of comparison we will also use this range of wavelengths. This range gives significantly smoother data.

In [ ]:
train_X, train_y, train_cat = X_y_cat(train_data,min_X=684,max_X=990)
test_X, test_y, test_cat = X_y_cat(test_data,min_X=684,max_X=990)
print(f'Number of selected spectra variables: {train_X.shape[1]}')

ax=train_X.mean().plot(label='Mean',legend=True)
for i in range(0,4):
    train_X.iloc[i,:].plot(label=i,legend=True)

## Expoloratory Data Analysis

We now dive into all the non-spectra data present in the dataset. We do this for the training data as we want to avoid learning about the test data set for now. We look at distributions of the target and the mean spectra for each categorical variable.

The **Set** variable deliminates data into training (cal), tuning (tuning) and test (val ext) sets. Previously we used this variable by combining the cal and tuning sets into a combined training data set. Each set has near identical mean spectra; indicating that these splits can be replaced by any random sampling method. DM distributions vary slightly.

In [ ]:
train_data['DM'].hist(by=train_data['Set'],bins=30,sharey=True,sharex=True,density =True)
train_data.groupby('Set')[train_X.columns].mean().transpose().plot()
train_data['Set'].value_counts()

Data is taken over 4 growing **Seasons**. The 4th season is used for the test set, with seasons 1-3 grouped and then divided into training and tuning sets. As for set, pectra readings from season are consistent, while DM varies slightly.

In [ ]:
train_data['DM'].hist(by=train_data['Season'],bins=30,sharey=True,sharex=True,density =True)
train_data.groupby('Season')[train_X.columns].mean().transpose().plot()
train_data['Season'].value_counts()

The two growing **Regions**, NT and QD also display similar spectra readings with slihgtly different DM distributions.

In [ ]:
train_data['DM'].hist(by=train_data['Region'],bins=30,sharey=True,sharex=True,density =True)
train_data.groupby('Region')[train_X.columns].mean().transpose().plot()
train_data['Region'].value_counts()

The Physiological stage, or **Type**, Hard Green or Ripened significantly influences spectra readings and ripened fruit having a much tighter DM distribution.

In [ ]:
train_data['DM'].hist(by=train_data['Type'],bins=30,sharey=True,sharex=True,density =True)
train_data.groupby('Type')[train_X.columns].mean().transpose().plot()
train_data['Type'].value_counts()

Each **Cultivar** displays a slightly different mean spectrs; cultivars display more variance at the low end and converge towards the high end. DM distributins vary highly.

In [ ]:
train_data['DM'].hist(by=train_data['Cultivar'],bins=30,sharey=True,sharex=True,density =True)
train_data.groupby('Cultivar')[train_X.columns].mean().transpose().plot()
train_data['Cultivar'].value_counts()

Mangoes are taken from 94 different **Populations** (orchards). This variable imacts spectra readings to a greater degree than cultivar. Its likely that there is significant overlap between the cultivar and population variables. 

In [ ]:
#train_data['DM'].hist(by=train_data['Pop'],bins=10,sharey=True,sharex=True)
train_data.groupby('Pop')[train_X.columns].mean().transpose().plot(legend=False)
train_data['Pop'].value_counts()

Readings at different **Temperature** show the same spectra reading. DM results are identical for each temeperature, which is expected as they share the same lab based label.

In [ ]:
train_data['DM'].hist(by=train_data['Temp'],bins=30,sharey=True,sharex=True,density =True)
train_data.groupby('Temp')[train_X.columns].mean().transpose().plot()
train_data['Temp'].value_counts()

## Explaining Variance

Having examined each categorical variable in turn we now perform a simple variance analysis. We build a linear model related DM to categorical variables. This is done for each combination of categorical variable (with temperature and date excluded), so that by using $R^2$ scores we can calculate the marginal contribution (to percent of variance explained) by each categorical variable.

Iterating through each combination of variables, we see that Pop explains 0.5474% of the variance. Including Cultivar adds an extra 0.002% for a toal of 0.5476%. We conclude that the effect of Region, Type, and Season variables are solely due to aggregates of Pop, or conversely, that the Pop variable aborbs all of the information of these variables.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from itertools import combinations

for i in range(1,6):
    oh_vars=['Season','Region','Type','Cultivar','Pop']
    for selected in list(combinations(oh_vars,i)):
        selected = list(selected)
        enc = OneHotEncoder()
        enc.fit(train_data[selected])
        oh_vars1 = enc.transform(train_data[selected])

        lin = LinearRegression()
        lin.fit(oh_vars1,train_y)
        score = lin.score(oh_vars1,train_y)

        print(f'R2 score is {score:.4f} for {selected}')

To further analyse this we group the data by pop. With the exception of Pop 52, which contains both KP, and LadyG cultivar, each population includes just a single unique value for each categorical value.
Variables like Region or Cultivar which may well effect dry matter, we just cannot measure this as their variance is absorbed by Pop. If we wanted to compare say Cultivar and Pop we would need a dataset containing multiple cultivars for each population.

In [ ]:
mangoes.groupby('Pop').agg({cat_var : 'unique' for cat_var in train_cat})

## A Linear Model and Summary

This notebook has introduced the mangoes dataset and performed simple expoloratory analysis, finding that the Pop variable explains over half the variance in the dataset.  This has implications for test set performance, with the training and test sets consisting of distinct sets of populations. 

In the next few notebooks we will get into the meat and potatoes of building regression models. To finish this notebook off we train a linear model on th training portion of the data, which gives an MSE score of 1.1290 on the test set.



In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
model = LinearRegression()
model.fit(train_X,train_y)
preds = model.predict(test_X)
mse = mean_squared_error(test_y,preds)
print(f"Test set MSE is: {mse:.4f}")